In [1]:
%%bash
# llamafile setup

# Step 1: Download a llamafile. The download may take several minutes.
# wget https://huggingface.co/Mozilla/Meta-Llama-3.1-8B-Instruct-llamafile/resolve/main/Meta-Llama-3.1-8B-Instruct.Q6_K.llamafile

# Step 2: Make the llamafile executable. Note: if you're on Windows, just append '.exe' to the filename.
# chmod +x Meta-Llama-3.1-8B-Instruct.Q6_K.llamafile

# Step 3: Start llamafile server in background. All the server logs will be written to 'tinyllama.log'.
# Alternatively, you can just open a separate terminal outside this notebook and run: 
#   ./Meta-Llama-3.1-8B-Instruct.Q6_K.llamafile --server --nobrowser --embedding
# ./Meta-Llama-3.1-8B-Instruct.Q6_K.llamafile --server --nobrowser --embedding > tinyllama.log 2>&1 &
./TinyLlama-1.1B-Chat-v1.0.Q5_K_M.llamafile --server --nobrowser --embedding > tinyllama.log 2>&1 &
pid=$!
echo "${pid}" > .llamafile_pid  # write the process pid to a file so we can terminate the server later

In [2]:
import uuid
from langchain_chroma import Chroma
import chromadb
from chromadb.config import Settings

from langchain_community.embeddings import LlamafileEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.llms.llamafile import Llamafile
from utils.text import get_pdf_text, get_text_chunks

In [3]:
client = chromadb.HttpClient(settings=Settings(allow_reset=True))
client.reset()  # resets the database

True

In [4]:
collection = client.create_collection("my_collection")

In [5]:
embedder = LlamafileEmbeddings()

In [6]:
llm = Llamafile()

In [7]:
docs = get_text_chunks(get_pdf_text("resume.pdf"))

In [8]:
vectorstore = openai_lc_client = Chroma.from_documents(
    docs, embedder, client=client, collection_name="my_collection"
)

In [9]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever())

In [10]:
query = "What professions did Gerson have?"
response = qa.run(query)
print(response)

/home/gerson/Projects/Magalu/vector-store/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


 Gerson had experience as a Data Scientist, Software Developer, and Project Manager.</s>


In [11]:
%%bash
# cleanup: kill the llamafile server process
kill $(cat .llamafile_pid)
rm .llamafile_pid